# Data Collector 
Для повышения модульности проект разбит на 2 блокнота - для сбора данных в датасет и собственно модель для работы с датасетом.  
Этот блокнот содержит сборщик данных

In [1]:
import numpy as np
import re
import pandas as pd
import glob
from html.parser import HTMLParser

import nltk
import nltk.tokenize as nt
import nltk.corpus as nc
import nltk.stem as ns
from nltk.stem import PorterStemmer,SnowballStemmer,LancasterStemmer
from nltk.tokenize.regexp import RegexpTokenizer

import pysrt

In [2]:
nltk.download('punkt')
nltk.download("stopwords")
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')
nltk.download('words')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Leonid\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [3]:
class MyHTMLParser(HTMLParser):
    text =""
    def handle_starttag(self, tag, attrs):
        pass


    def handle_endtag(self, tag):
        pass


    def handle_data(self, data):
        self.text+=data



Словарь загружается из текстоых файлов, каждый файл содержит словарные статьи с указанием частей речи.  
Словарь должен содержать слова одной категории, категория указывается при загрузке файла

In [4]:
class Vocabulary():
#    part_of_speech = ['n.','v.','adj.','conj.','prep.','pron.']
#    template = parts_of_speech.join('')
    
    def __init__(self, filenames):
        self.words = {}
        for fn,grade in filenames.items():
            with open(fn) as f:
                for ln in f:
                    w,set_g = self.split_line(ln)
                    self.insert(word = w,grade = grade, grammar= set_g )
                            
    def split_line(self,ln):
        template='((n)|(v)|(prep)|(pron)|(conj)|(adj)|(adv))[\.\,]?'
        tokens =ln.split(' ')
        word = tokens[0]
        gram = set()
        for t in tokens[1:]:
            m = re.match(template,t) 
            if m:
                gram.add(m[1])
        return word,gram
                            
    def insert(self, word, grade, grammar):
        ww = self.words.pop(word,{})
        gr = ww.pop('grammar',set())
        gr |= grammar
        ww['grammar'] =  gr
        g = ww.pop('grade',set())
        g.add(grade)
        ww['grade']=g
        self.words[word] = ww 
        
    def summary(self):
        count= 0
        grades = {}
        grammars = {}
        for v in self.words.values():
            if not str(v['grammar']) in grammars:
                grammars[str(v['grammar'])] = 0 
            grammars[str(v['grammar'])]+=1
            if not str(v['grade']) in grades:
                grades[str(v['grade'])] = 0 
            grades[str(v['grade'])]+=1
            count+=1    
        return count,grades,grammars    


Простая обёртка для загрузки и чистки стоп-слов

In [5]:
class Stops:
    def __init__(self ):
        self.stop_words = set(nc.stopwords.words("english")) | {"'"}
    
    def purge(self,words): 
        return(  [ w for w in words if w.casefold() not in self.stop_words ] )

stops = Stops()    

Класс для сбора собственных имён, при анализе частота собственных имён является отдельным признаком 

In [6]:
class NE_Chunks:
    def __init__(self):
        pass 
    
    def extract(self,words):
        tags = nltk.pos_tag(words)
        tree = nltk.ne_chunk(tags, binary=True)
        ne =  set(
                " ".join(i[0] for i in t)
                for t in tree
                if hasattr(t, "label") and t.label() == "NE"
            )
        common = words #[ w for w in words if w  not in ne ]  
        return(ne,common)
    
ne_chunks = NE_Chunks()    

In [ ]:
На прикидках LancasterStemmer показал полное превосходство над остальными, не нашёл, в чём он может уступать

In [7]:
#stemmer = SnowballStemmer('english')
stemmer = LancasterStemmer()

Класс для сбора статистики с файлов с субтитрами.  
Подсчитывает частотность слов каждой категории по использванию и по доле в словаре.   
Также вычисляется общее количество слов, средняя дляни слова, частотность собственных имен.
Порядка 30 % слов не попадает в подсчет (баг пока не найден), но по этой причене можно считать, что частотности категорий не коррелируют между собой :) 

In [8]:
class Subfile():

    def __init__(self,filename):
        self.text = ''
        self.ne ={}
        self.freq = dict()
        self.summary = dict()
        self.tokens = list()
        file_name_parts = filename.split('/')[-1].split('\\')[-1].split('.')
        self.movie = '.'.join( file_name_parts[:-1])                           # w/o 'srt'
        self.misses = {}
#        print(self.movie)
        self.load(filename)
    
    def load(self,filename):
        encs = ['utf-8','iso-8859-1']
        last_error = None
        for en in encs:
            try:
                srt = pysrt.open(filename, encoding=en )
            except UnicodeDecodeError as e:
                last_error = e
                srt = []
            if srt :
                break
        
        if not srt:
            raise last_error 
        h=MyHTMLParser()
        h.feed(srt.text)
        self.text = h.text         
    
    def parse(self):
#        self.tokens = nt.word_tokenize(self.text)
        self.tokens = stops.purge( self.tokenize(self.text) )
        self.ne,self.tokens =  ne_chunks.extract(self.tokens)
        self.tokens = [w.lower() for w in self.tokens ]
        self.ne = {w.lower() for w in self.ne }
        self.summary = nltk.FreqDist(self.tokens)
        
    def tokenize(self,text):
        rt = RegexpTokenizer(r"\w+|\'")
        return( rt.tokenize(text) ) 
    
    
    def classify(self,vocab):
        def find_word_grade(w):
            gr = {}
            try:
                if w in vocab.words  : 
                    gr = max(vocab.words[w]['grade'])
                elif  (stemmer.stem(w) in vocab.words):    
                    gr= max(vocab.words[stemmer.stem(w)]['grade'])
            except ValueError as ve: 
                print(w,ve)
                
            return(gr) 
    
        def inc_dict_at( d,i,n):
            g = d.get(i,0)
            d[i] = g+n
  
                                      
        cnt = 0
        cnt_ne = 0
        sum_length = 0 
        nums = dict()
        self.misses = {}
        ids = dict()
        g = {}
        for w,n in self.summary.items():
#             print(w)
            g = find_word_grade(w)
            if bool(g):
                inc_dict_at( nums,g,n)
                inc_dict_at( ids, g,1)
            elif w in self.ne:
                cnt_ne += 1
            else:    
                self.misses[w]=n                                     
            cnt += n 
            sum_length += len(w)

        self.freq = {  k+'_freq':round(v/cnt,4) for (k,v)  in nums.items() }
        wc = len( self.summary ) 
        self.freq |= { k+'_rate':round(v/wc,4) for (k,v)  in ids.items() }
        self.freq |= { 'qty': round(cnt), 'ne': round(cnt_ne/cnt,4), 'avg_length':round(sum_length/cnt,1) }

    def statistic(self):
        return({'movie':self.movie} | self.freq )
    

Сборщик статистики по всем файлам , сохраняет результаты в датасет в каталоге dataset

In [9]:
class DataCollector:

    def __init__(self): #, srt_pattern , scores_path)
#         self.srt_pattern= srt_pattern
#         self.xls_path = scores_path 
        self.process()
    
    def get_levels(self,xls_path):
        df = pd.read_excel(xls_path)
        df = df.drop('id',axis=1)
        df.columns = ['movie','level'] 
        resolve_levels = {'A2/A2+':'A2' ,'A2/A2+':'A2','A2/A2+, B1':'A2','B1, B2':'B1' }
        df['level'] = df.level.apply(lambda l: resolve_levels.get( l.strip(),l.strip() ) )
        return df

    def gather_stats(self,file_pattern,vocab):
        def add_one(acc, new ):
            for k in acc:
                if k not in new:
                    new[k]=0
            for k,v in new.items():
                if k in acc:
                    acc[k].append(v)
                else:
                    acc[k] = [v]

        acc = {}
        for fn in glob.glob(file_pattern):
            try:
                sf= Subfile(fn)        
                sf.parse()
            except UnicodeDecodeError as e:
        #        print(sf.movie,e)
                continue
            sf.classify(vocab = vocab)
            st = sf.statistic()
            add_one(acc,st)
        #    print(acc.keys(), st.keys())
        return( pd.DataFrame.from_dict( acc ) ) 
    
    def join_data(self, ds , df ):
        dx = ds.merge(df,how='outer')
        return(dx)
        
    def write(self, dx ):
        dx.to_csv('datasets/movies.csv')
        with open("movies_params.py", "w") as w:
            w.write("""\
#!/usr/bin/env python
# coding: utf-8
        """ )
            w.write("\n")
            w.write(f"feature_cols = {self.feature_cols(dx)} \n")
            w.write(f"target_col = 'level'\n")   
            
    def feature_cols(self,dx):
        return( [c for c in dx.columns if c != 'movie'] )
            
    def process(self):
        vocab =  Vocabulary({ 'Oxford_CEFR_level/A1.txt':'A1',
                       'Oxford_CEFR_level/A2.txt':'A2',
                      'Oxford_CEFR_level/B1.txt':'B1', 
                      'Oxford_CEFR_level/B2.txt':'B2',
                      'Oxford_CEFR_level/C1.txt':'C1'   } ) 
        df = self.get_levels('English_scores/movies_labels.xlsx')
        ds = self.gather_stats( 'English_scores/Subtitles_all/*/*.srt', vocab )
        dx = self.join_data(ds,df)
        self.write(dx)
         

In [10]:
%%time
DataCollector()

CPU times: total: 3min 5s
Wall time: 3min 15s
